# Rekonstruktion einer Baumstruktur mit pc-skeletor
Dieses Notebook verwendet das GitHub-Projekt [pc-skeletor](https://github.com/meyerls/pc-skeletor), um aus einer 3D-Punktwolke das topologische Skelett zu extrahieren. Der Algorithmus basiert auf einer Voxelisierung, Distanztransformation und Graphanalyse.

## Voraussetzungen
- Python ≥ 3.6
- GitHub-Repo: https://github.com/meyerls/pc-skeletor
- Installation:
```bash
git clone https://github.com/meyerls/pc-skeletor.git
cd pc-skeletor
pip install -e .
```

## 1. Punktwolke einlesen (echter Baum)

In [ ]:
import numpy as np
import pandas as pd

# Punktwolke als CSV (x, y, z-Spalten)
data = pd.read_csv("your_tree_pointcloud.csv")  # <- Datei anpassen
points = data[['x', 'y', 'z']].to_numpy()
print("Punktanzahl:", points.shape)


## 2. Skeletonisierung mit pc-skeletor

In [ ]:
from skeletor import Skeletonizer
from skeletor.voxelize import voxelize

# Voxelgröße (abhängig von Baumgröße anpassen)
voxel_size = 0.05

# Voxelisiere Punktwolke
volume, origin, spacing = voxelize(points, voxel_size=voxel_size)

# Skeletonizer initialisieren und ausführen
skel = Skeletonizer(volume)
skel.skeletonize(method='zhang')  # andere Methoden: 'medial', 'thinning'
graph = skel.to_graph(min_branch_length=2)

print(f"Anzahl Knoten im Skelett: {graph.number_of_nodes()}")


## 3. Visualisierung des extrahierten Skeletts

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(111, projection='3d')

# Originalpunkte (optional)
ax.scatter(points[:,0], points[:,1], points[:,2], s=1, alpha=0.05)

# Skelettpunkte plotten
for u, v in graph.edges():
    p1 = np.array(graph.nodes[u]['coord']) * spacing + origin
    p2 = np.array(graph.nodes[v]['coord']) * spacing + origin
    ax.plot([p1[0], p2[0]], [p1[1], p2[1]], [p1[2], p2[2]], 'r-', linewidth=1)

ax.set_title("Topologisches Skelett des Baums (pc-skeletor)")
plt.show()
